In [1]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")


In [2]:
#embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#embed_model_id = 'voyage-lite-02-instruct' #not available through HF
#embed_model_id = 'WhereIsAI/UAE-Large-V1'
embed_model_id = 'mixedbread-ai/mxbai-embed-large-v1'
core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={"trust_remote_code":True}
    # model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id

)

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

In [3]:
import os
existing_files = set(os.listdir('../abstracts'))

In [29]:
#existing_files = list(existing_files)

In [4]:
def get_title_abstract_abbrev(directory = '../abstracts/',filename=None):
    path = os.path.join(directory, filename)
    with open(path, 'r') as file:
        lines = file.readlines()
        title = lines[0].strip()
        abstract = "".join(lines[1:]).replace('\n', ' ')
    return title, abstract, filename.replace('.txt','')


In [5]:
get_title_abstract_abbrev(filename=next(iter(existing_files)))

('Chain-of-Thought Prompting Elicits Reasoning in Large Language Models',
 'We explore how generating a chain of thought -- a series of intermediate reasoning steps -- significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain of thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting. Experiments on three large language models show that chain of thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a 540B-parameter language model with just eight chain of thought exemplars achieves state of the art accuracy on the GSM8K benchmark of math word problems, surpassing even finetuned GPT-3 with a verifier.',
 '2201.11903')

In [6]:
for file in existing_files:
    title, abstract,_ = get_title_abstract_abbrev(filename=file)
    print(title)
    print(abstract)
    print()

Chain-of-Thought Prompting Elicits Reasoning in Large Language Models
We explore how generating a chain of thought -- a series of intermediate reasoning steps -- significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain of thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting. Experiments on three large language models show that chain of thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a 540B-parameter language model with just eight chain of thought exemplars achieves state of the art accuracy on the GSM8K benchmark of math word problems, surpassing even finetuned GPT-3 with a verifier.

KTO: Model Alignment as Prospect Theoretic Optimization
Kahneman 

In [7]:
import pandas as pd
new_articles=pd.read_csv('/home/mainuser/Desktop/LLMs/RagOverArXiv/scrape/data/articles_up_to_2024-04-16.csv')

In [8]:
new_articles.head()

,title,authors,abstract,arxiv_abbrev
0,Ferret-UI: Grounded Mobile UI Understanding wi...,"Authors:Keen You,Haotian Zhang,Eldon Schoop,Fl...",Recent advancements in multimodal large langua...,2404.05719
1,"ByteEdit: Boost, Comply and Accelerate Generat...","Authors:Yuxi Ren,Jie Wu,Yanzuo Lu,Huafeng Kuan...",Recent advancements in diffusion-based generat...,2404.04860
2,SpatialTracker: Tracking Any 2D Pixels in 3D S...,"Authors:Yuxi Xiao,Qianqian Wang,Shangzhan Zhan...",Recovering dense and long-range pixel motion i...,2404.04319
3,SwapAnything: Enabling Arbitrary Object Swappi...,"Authors:Jing Gu,Yilin Wang,Nanxuan Zhao,Wei Xi...",Effective editing of personal content holds a ...,2404.05717
4,BeyondScene: Higher-Resolution Human-Centric S...,"Authors:Gwanghyun Kim,Hayeon Kim,Hoigi Seo,Don...",Generating higher-resolution human-centric sce...,2404.04544


In [35]:
from langchain.schema.document import Document

In [36]:
docs = [get_title_abstract_abbrev(filename=file) for file in existing_files]
docs = [Document(page_content=doc[1],metadata={'title':doc[0], 'abbrev':doc[2]}) for doc in docs]


In [37]:
len(docs)

37

In [12]:
# dir(docs[0])

In [13]:
docs[0].__getattribute__('page_content')

'We explore how generating a chain of thought -- a series of intermediate reasoning steps -- significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain of thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting. Experiments on three large language models show that chain of thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a 540B-parameter language model with just eight chain of thought exemplars achieves state of the art accuracy on the GSM8K benchmark of math word problems, surpassing even finetuned GPT-3 with a verifier.'

In [14]:
len(docs)

37

- TODO: using the merge functionality, write a function to load an existing vector store and add a new vector to it

In [38]:
# for i, doc in enumerate(docs):
#    #doc = doc[1]
#    #content = docs[index]
#    if i == 0:
#        vector_store = FAISS.from_documents([doc], embedder)
#    else:
#       vector_store_i = FAISS.from_documents([doc], embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store

In [39]:
# Adding vs_index: Note if add new ones, to existing local vs, get len(vector_store.index_to_docstore_id)
# and start from there
for i, doc in enumerate(docs):
   doc.metadata['vs_index'] = i
   if i == 0:
       vector_store = FAISS.from_documents([doc], embedder)
   else:
      vector_store_i = FAISS.from_documents([doc], embedder)
      vector_store.merge_from(vector_store_i)

vector_store

In [29]:
#dir(vector_store)

In [40]:
len(vector_store.index_to_docstore_id)

37

In [41]:
vector_store.index_to_docstore_id

{0: '35b265b4-aeac-45a2-a6b9-6cd6f2cd4a0e',
 1: 'b8066f6c-0539-4027-9913-f59fdd35f34c',
 2: 'd5ebcb60-b3c5-4972-93fc-e6d94e155a82',
 3: 'ceee7f48-574c-4648-a0c7-7d47ece94b10',
 4: 'f2386332-2b03-43f0-a5be-6d5956d2de93',
 5: '4b812aa4-91fe-4351-a32e-e8c48bc83b89',
 6: 'f024d7a7-554e-4b82-ac7f-6b262c81bcb4',
 7: 'de15d716-e042-47da-bdc5-884a4eff0464',
 8: '6e4497be-cbf8-4301-b5b5-cc23631d3c74',
 9: '1d6d9d17-72a0-4607-b32d-db8c0c601557',
 10: '83e62da9-1041-46f5-baac-162ccd44728f',
 11: '342e9ced-f243-4fe0-bd2a-3ccfa8af48b3',
 12: '6e07a052-3945-4d39-a3da-62942503010f',
 13: '64427e3d-fa77-408d-839c-1e17168c51ce',
 14: 'f58c7495-e375-4035-b3f2-874a96ae0a87',
 15: '03329430-08cb-41e3-b756-df99a13f8c3e',
 16: 'b7ae1856-79fd-4dc2-8348-d354986eb3e5',
 17: '3624a876-f214-4595-b95e-61f2dcfff9fc',
 18: 'fe9fb227-18e8-4988-adda-9dbfee1f452e',
 19: '5a42a887-4c5f-4c3f-bdaf-076d39cf78f4',
 20: 'f8c6c3cc-2e5f-4946-b620-8d10d0a099a0',
 21: 'cebebc57-820c-4628-b8c7-a5b7b09aff45',
 22: '02ab3468-d26a-

In [42]:
index_id = 36
embedding_vector = vector_store.index.reconstruct_n(index_id, 1)[0]

In [43]:
embedding_vector.shape

(1024,)

In [44]:
docs[36]

Document(page_content="Language models are increasingly being deployed for general problem solving across a wide range of tasks, but are still confined to token-level, left-to-right decision-making processes during inference. This means they can fall short in tasks that require exploration, strategic lookahead, or where initial decisions play a pivotal role. To surmount these challenges, we introduce a new framework for language model inference, Tree of Thoughts (ToT), which generalizes over the popular Chain of Thought approach to prompting language models, and enables exploration over coherent units of text (thoughts) that serve as intermediate steps toward problem solving. ToT allows LMs to perform deliberate decision making by considering multiple different reasoning paths and self-evaluating choices to decide the next course of action, as well as looking ahead or backtracking when necessary to make global choices. Our experiments show that ToT significantly enhances language model

In [23]:
#vector_store.asimilaritysearch('What is PPO?')
vector_store.asimilarity_search('What is PPO?')

<coroutine object FAISS.asimilarity_search at 0x7346eeee56c0>

In [45]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
#query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
#query = ''
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)
  print(page.metadata)

While large-scale unsupervised language models (LMs) learn broad world knowledge and some reasoning skills, achieving precise control of their behavior is difficult due to the completely unsupervised nature of their training. Existing methods for gaining such steerability collect human labels of the relative quality of model generations and fine-tune the unsupervised LM to align with these preferences, often with reinforcement learning from human feedback (RLHF). However, RLHF is a complex and often unstable procedure, first fitting a reward model that reflects the human preferences, and then fine-tuning the large unsupervised LM using reinforcement learning to maximize this estimated reward without drifting too far from the original model. In this paper we introduce a new parameterization of the reward model in RLHF that enables extraction of the corresponding optimal policy in closed form, allowing us to solve the standard RLHF problem with only a simple classification loss. The resu

In [46]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
source = 'Learn Your Reference Model for Real Good Alignment'
abbrev = '2404.09656'
new_doc = Document(page_content=query,metadata={'source':source, 'abbrev':abbrev})
vector_store_i = FAISS.from_documents([new_doc], embedder)
vector_store.merge_from(vector_store_i)

In [47]:
len(vector_store.index_to_docstore_id)

38

In [48]:
# Experimental
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
#query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
#query = ''
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 10)

for page in docs:
  print(page.page_content)
  print(page.metadata)

The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straig

In [49]:
# query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
# #query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
# #query = ''
# embedding_vector = core_embeddings_model.embed_query(query)
# docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

# for page in docs:
#   print(page.page_content)
#   print(page.metadata)

In [51]:
new_articles.loc[0,'abstract']

'Recent advancements in multimodal large language models (MLLMs) have been noteworthy, yet, these general-domain MLLMs often fall short in their ability to comprehend and interact effectively with user interface (UI) screens. In this paper, we present Ferret-UI, a new MLLM tailored for enhanced understanding of mobile UI screens, equipped with referring, grounding, and reasoning capabilities. Given that UI screens typically exhibit a more elongated aspect ratio and contain smaller objects of interest (e.g., icons, texts) than natural images, we incorporate "any resolution" on top of Ferret to magnify details and leverage enhanced visual features. Specifically, each screen is divided into 2 sub-images based on the original aspect ratio (i.e., horizontal division for portrait screens and vertical division for landscape screens). Both sub-images are encoded separately before being sent to LLMs. We meticulously gather training samples from an extensive range of elementary UI tasks, such as

In [52]:
query=new_articles.loc[0,'abstract']
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 10)

for page in docs:
  print(page.page_content)
  print(page.metadata)

We continue the investigation into the power of smaller Transformer-based language models as initiated by \textbf{TinyStories} -- a 10 million parameter model that can produce coherent English -- and the follow-up work on \textbf{phi-1}, a 1.3 billion parameter model with Python coding performance close to the state-of-the-art. The latter work proposed to use existing Large Language Models (LLMs) to generate ``textbook quality" data as a way to enhance the learning process compared to traditional web data. We follow the ``Textbooks Are All You Need" approach, focusing this time on common sense reasoning in natural language, and create a new 1.3 billion parameter model named \textbf{phi-1.5}, with performance on natural language tasks comparable to models 5x larger, and surpassing most non-frontier LLMs on more complex reasoning tasks such as grade-school mathematics and basic coding. More generally, \textbf{phi-1.5} exhibits many of the traits of much larger LLMs, both good -- such as 

In [53]:
new_articles.loc[1,'abstract']

"Recent advancements in diffusion-based generative image editing have sparked a profound revolution, reshaping the landscape of image outpainting and inpainting tasks. Despite these strides, the field grapples with inherent challenges, including: i) inferior quality; ii) poor consistency; iii) insufficient instrcution adherence; iv) suboptimal generation efficiency. To address these obstacles, we present ByteEdit, an innovative feedback learning framework meticulously designed to Boost, Comply, and Accelerate Generative Image Editing tasks. ByteEdit seamlessly integrates image reward models dedicated to enhancing aesthetics and image-text alignment, while also introducing a dense, pixel-level reward model tailored to foster coherence in the output. Furthermore, we propose a pioneering adversarial and progressive feedback learning strategy to expedite the model's inference speed. Through extensive large-scale user evaluations, we demonstrate that ByteEdit surpasses leading generative im

In [56]:
query=new_articles.loc[0,'abstract']
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 10)
vs_indices = [doc.metadata['vs_index'] for doc in docs]
for page in docs:
  print(page.page_content)
  print(page.metadata)

We continue the investigation into the power of smaller Transformer-based language models as initiated by \textbf{TinyStories} -- a 10 million parameter model that can produce coherent English -- and the follow-up work on \textbf{phi-1}, a 1.3 billion parameter model with Python coding performance close to the state-of-the-art. The latter work proposed to use existing Large Language Models (LLMs) to generate ``textbook quality" data as a way to enhance the learning process compared to traditional web data. We follow the ``Textbooks Are All You Need" approach, focusing this time on common sense reasoning in natural language, and create a new 1.3 billion parameter model named \textbf{phi-1.5}, with performance on natural language tasks comparable to models 5x larger, and surpassing most non-frontier LLMs on more complex reasoning tasks such as grade-school mathematics and basic coding. More generally, \textbf{phi-1.5} exhibits many of the traits of much larger LLMs, both good -- such as 

In [57]:
vs_indices

[15, 33, 34, 35, 32, 4, 13, 17, 6, 28]

In [58]:
import numpy as np
import torch
#similar_embedding_vectors = torch.tensor(np.array([vector_store.index.reconstruct_n(index_id, 1)[0] for index_id in vs_indices]))
similar_embedding_vectors = np.array([vector_store.index.reconstruct_n(index_id, 1)[0] for index_id in vs_indices])

In [59]:
similar_embedding_vectors.shape

(10, 1024)

In [60]:
np.array(embedding_vector).shape

(1024,)

In [61]:
distances = np.linalg.norm(similar_embedding_vectors-np.array(embedding_vector), axis=1)
average_distance = np.mean(distances)
distances, average_distance

(array([12.20101666, 12.42112783, 12.57575874, 12.70079431, 12.72302538,
        12.77311078, 12.8005531 , 12.80305586, 12.96793891, 13.0623732 ]),
 12.702875475669595)

- 12.7 avg similarity for TinyStories paper and 14.7 for diffusion one (of which don't have many in Zotero), seems sensible
- Could perhaps take top-1 as similarity measure (read that paper, after all) and average_distance from my vector to all the embedding vectors as novelty, combine these two...

- OK, so the issue is that the papers are a bit over the map, generally in ['llms','vision','diffusion','other camp'], could try to you use a smaller quanitized model to extract topic, just need to monitor the time taken.

In [62]:
from exllamav2 import *
from exllamav2.generator import *
import sys, torch


generator_config = ExLlamaV2Config()
generator_config.model_dir = "/home/mainuser/Desktop/LLMs/MiStralInference"
#generator_config.model_dir = '/home/mainuser/Desktop/LLMs/Mixtral4bit'
generator_config.prepare()

generator_model = ExLlamaV2(generator_config)
cache = ExLlamaV2Cache(generator_model, lazy = True)

print("Loading model...")
generator_model.load_autosplit(cache)

generator_tokenizer = ExLlamaV2Tokenizer(generator_config)
generator_llm = ExLlamaV2StreamingGenerator(generator_model, cache, generator_tokenizer)
generator_llm.set_stop_conditions([generator_tokenizer.eos_token_id])
generator_settings = ExLlamaV2Sampler.Settings()
generator_settings.temperature = 0.85
generator_settings.top_k = 50
generator_settings.top_p = 0.8
generator_settings.token_repetition_penalty = 1.01

Loading model...


In [63]:
def call_llm(
    question: str,
    generator: ExLlamaV2StreamingGenerator,
    settings:ExLlamaV2Sampler.Settings,
    max_new_tokens = 512
    ):

    max_new_tokens = max_new_tokens

    generator.warmup()
    output = generator.generate_simple(f"<s>[INST] {question} [/INST]", settings, max_new_tokens, seed = 1234)
    return output




In [64]:
abstract = new_articles.loc[0,'abstract']
topic_classification_prompt = """
Your task is to take an arXiv abstract and classify it into one of the following categories: LLMs, stable diffusion, computer vision, or other.
Only classify the abstract as 'other' if you're sure it doesn't fit into any of the other categories.

Provide your answer as follows:


Answer: category

Now here is the abstract: {abstract}

Output:::"""
#topic_classification_prompt = 
call_llm(question=topic_classification_prompt, generator=generator_llm,settings=generator_settings,max_new_tokens=24)

"<s>[INST] \nYour task is to take an arXiv abstract and classify it into one of the following categories: LLMs, stable diffusion, computer vision, or other.\nOnly classify the abstract as 'other' if you're sure it doesn't fit into any of the other categories.\n\nProvide your answer as follows:\n\n\nAnswer: category\n\nNow here is the abstract: {abstract}\n\nOutput::: [/INST] I'm sorry, but I need the abstract to classify it into a category. Please provide the abstract."

In [65]:
abstract

'Recent advancements in multimodal large language models (MLLMs) have been noteworthy, yet, these general-domain MLLMs often fall short in their ability to comprehend and interact effectively with user interface (UI) screens. In this paper, we present Ferret-UI, a new MLLM tailored for enhanced understanding of mobile UI screens, equipped with referring, grounding, and reasoning capabilities. Given that UI screens typically exhibit a more elongated aspect ratio and contain smaller objects of interest (e.g., icons, texts) than natural images, we incorporate "any resolution" on top of Ferret to magnify details and leverage enhanced visual features. Specifically, each screen is divided into 2 sub-images based on the original aspect ratio (i.e., horizontal division for portrait screens and vertical division for landscape screens). Both sub-images are encoded separately before being sent to LLMs. We meticulously gather training samples from an extensive range of elementary UI tasks, such as

In [66]:
import re

In [255]:
abstract = new_articles.loc[0,'abstract']
topic_classification_prompt = f"""
Your task is to take an arXiv abstract and classify it into one of the following categories: LLMs, diffusion, computer vision, or other.
Only classify the abstract as 'other' if you're sure it doesn't fit into any of the other categories.

Provide your answer as follows:


Answer: category

Now here is the abstract: {abstract}

Output::: """
#topic_classification_prompt = 
ans = call_llm(question=topic_classification_prompt, generator=generator_llm,settings=generator_settings,max_new_tokens=48)[len(topic_classification_prompt):]


In [68]:
ans

'Output:::  [/INST] Category: LLMs'

In [69]:
pattern = r'(Category|Answer):\s*(\w+)'
match = re.search(pattern, ans)
match.group(2)

'LLMs'

In [256]:
# #abstract = r"""JWST is discovering a large population of z>4 supermassive black holes (SMBHs) that are overmassive with respect to the stellar content of their hosts. A previous study developed a physical model to interpret this overmassive population as the result of quasar feedback acting on a compact host galaxy. In this Note, we apply this model to JADES GN 1146115, a dormant supermassive black hole at z=6.7 whose mass is ∼40% of the host's mass in stars and accreting at ∼2% of the Eddington limit. The host has been forming stars at the low rate of ∼1M⊙yr−1 for the past ∼100 Myr. Our model suggests that this galactic system is on the verge of a resurgence of global star formation activity. This transition comes after a period of domination by the effect of its overmassive black hole, whose duration is comparable to typical quasar lifetimes. """
# #abstract = r"""Text animation serves as an expressive medium, transforming static communication into dynamic experiences by infusing words with motion to evoke emotions, emphasize meanings, and construct compelling narratives. Crafting animations that are semantically aware poses significant challenges, demanding expertise in graphic design and animation. We present an automated text animation scheme, termed "Dynamic Typography", which combines two challenging tasks. It deforms letters to convey semantic meaning and infuses them with vibrant movements based on user prompts. Our technique harnesses vector graphics representations and an end-to-end optimization-based framework. This framework employs neural displacement fields to convert letters into base shapes and applies per-frame motion, encouraging coherence with the intended textual concept. Shape preservation techniques and perceptual loss regularization are employed to maintain legibility and structural integrity throughout the animation process. We demonstrate the generalizability of our approach across various text-to-video models and highlight the superiority of our end-to-end methodology over baseline methods, which might comprise separate tasks. Through quantitative and qualitative evaluations, we demonstrate the effectiveness of our framework in generating coherent text animations that faithfully interpret user prompts while maintaining readability. Our code is available at: https://animate-your-word.github.io/demo/."""
# #abstract = r"""We introduce Reka Core, Flash, and Edge, a series of powerful multimodal language models trained from scratch by Reka. Reka models are able to process and reason with text, images, video, and audio inputs. This technical report discusses details of training some of these models and provides comprehensive evaluation results. We show that Reka Edge and Reka Flash are not only state-of-the-art but also outperform many much larger models, delivering outsized values for their respective compute class. Meanwhile, our most capable and largest model, Reka Core, approaches the best frontier models on both automatic evaluations and blind human evaluations. On image question answering benchmarks (e.g. MMMU, VQAv2), Core performs competitively to GPT4-V. Meanwhile, on multimodal chat, Core ranks as the second most preferred model under a blind third-party human evaluation setup, outperforming other models such as Claude 3 Opus. On text benchmarks, Core not only performs competitively to other frontier models on a set of well-established benchmarks (e.g. MMLU, GSM8K) but also outperforms GPT4-0613 on human evaluation. On video question answering (Perception-Test), Core outperforms Gemini Ultra. Models are shipped in production at http://chat.reka.ai . A showcase of non cherry picked qualitative examples can also be found at http://showcase.reka.ai ."""
# abstract = r"""Despite the impressive capabilities of Large Language Models (LLMs) on various tasks, they still struggle with scenarios that involves complex reasoning and planning. Recent work proposed advanced prompting techniques and the necessity of fine-tuning with high-quality data to augment LLMs' reasoning abilities. However, these approaches are inherently constrained by data availability and quality. In light of this, self-correction and self-learning emerge as viable solutions, employing strategies that allow LLMs to refine their outputs and learn from self-assessed rewards. Yet, the efficacy of LLMs in self-refining its response, particularly in complex reasoning and planning task, remains dubious. In this paper, we introduce AlphaLLM for the self-improvements of LLMs, which integrates Monte Carlo Tree Search (MCTS) with LLMs to establish a self-improving loop, thereby enhancing the capabilities of LLMs without additional annotations. Drawing inspiration from the success of AlphaGo, AlphaLLM addresses the unique challenges of combining MCTS with LLM for self-improvement, including data scarcity, the vastness search spaces of language tasks, and the subjective nature of feedback in language tasks. AlphaLLM is comprised of prompt synthesis component, an efficient MCTS approach tailored for language tasks, and a trio of critic models for precise feedback. Our experimental results in mathematical reasoning tasks demonstrate that AlphaLLM significantly enhances the performance of LLMs without additional annotations, showing the potential for self-improvement in LLMs."""

# topic_classification_prompt = f"""
# Your task is to take an arXiv abstract and classify it into one of the following categories: LLMs, diffusion, computer vision, multimodal.
# If you see multimodal in abstract, classify as multimodal.  If you see diffusion in the abstract, classify as diffusion.
# If you see LLMs or language models in the abstract, classify as LLMs.  If you see computer vision in the abstract, classify as computer vision.
# Now here is the abstract: {abstract}
# Check your answer before returning it and provide a brief explanation.


# Category:
# """
# #topic_classification_prompt = 
# ans = call_llm(question=topic_classification_prompt, generator=generator_llm,settings=generator_settings,max_new_tokens=24)[len(topic_classification_prompt):]
# #pattern = r'(Category|Answer):\s*(\w+)'
# pattern = r'(LLMs|diffusion|computer vision|multimodal)'
# match = re.search(pattern, ans,re.IGNORECASE)
# response =  match.group(1) if match else 'other'
# response

'LLMs'

In [278]:
#abstract = r"""JWST is discovering a large population of z>4 supermassive black holes (SMBHs) that are overmassive with respect to the stellar content of their hosts. A previous study developed a physical model to interpret this overmassive population as the result of quasar feedback acting on a compact host galaxy. In this Note, we apply this model to JADES GN 1146115, a dormant supermassive black hole at z=6.7 whose mass is ∼40% of the host's mass in stars and accreting at ∼2% of the Eddington limit. The host has been forming stars at the low rate of ∼1M⊙yr−1 for the past ∼100 Myr. Our model suggests that this galactic system is on the verge of a resurgence of global star formation activity. This transition comes after a period of domination by the effect of its overmassive black hole, whose duration is comparable to typical quasar lifetimes. """
#abstract = r"""Text animation serves as an expressive medium, transforming static communication into dynamic experiences by infusing words with motion to evoke emotions, emphasize meanings, and construct compelling narratives. Crafting animations that are semantically aware poses significant challenges, demanding expertise in graphic design and animation. We present an automated text animation scheme, termed "Dynamic Typography", which combines two challenging tasks. It deforms letters to convey semantic meaning and infuses them with vibrant movements based on user prompts. Our technique harnesses vector graphics representations and an end-to-end optimization-based framework. This framework employs neural displacement fields to convert letters into base shapes and applies per-frame motion, encouraging coherence with the intended textual concept. Shape preservation techniques and perceptual loss regularization are employed to maintain legibility and structural integrity throughout the animation process. We demonstrate the generalizability of our approach across various text-to-video models and highlight the superiority of our end-to-end methodology over baseline methods, which might comprise separate tasks. Through quantitative and qualitative evaluations, we demonstrate the effectiveness of our framework in generating coherent text animations that faithfully interpret user prompts while maintaining readability. Our code is available at: https://animate-your-word.github.io/demo/."""
#abstract = r"""We introduce Reka Core, Flash, and Edge, a series of powerful multimodal language models trained from scratch by Reka. Reka models are able to process and reason with text, images, video, and audio inputs. This technical report discusses details of training some of these models and provides comprehensive evaluation results. We show that Reka Edge and Reka Flash are not only state-of-the-art but also outperform many much larger models, delivering outsized values for their respective compute class. Meanwhile, our most capable and largest model, Reka Core, approaches the best frontier models on both automatic evaluations and blind human evaluations. On image question answering benchmarks (e.g. MMMU, VQAv2), Core performs competitively to GPT4-V. Meanwhile, on multimodal chat, Core ranks as the second most preferred model under a blind third-party human evaluation setup, outperforming other models such as Claude 3 Opus. On text benchmarks, Core not only performs competitively to other frontier models on a set of well-established benchmarks (e.g. MMLU, GSM8K) but also outperforms GPT4-0613 on human evaluation. On video question answering (Perception-Test), Core outperforms Gemini Ultra. Models are shipped in production at http://chat.reka.ai . A showcase of non cherry picked qualitative examples can also be found at http://showcase.reka.ai ."""
#abstract = r"""Despite the impressive capabilities of Large Language Models (LLMs) on various tasks, they still struggle with scenarios that involves complex reasoning and planning. Recent work proposed advanced prompting techniques and the necessity of fine-tuning with high-quality data to augment LLMs' reasoning abilities. However, these approaches are inherently constrained by data availability and quality. In light of this, self-correction and self-learning emerge as viable solutions, employing strategies that allow LLMs to refine their outputs and learn from self-assessed rewards. Yet, the efficacy of LLMs in self-refining its response, particularly in complex reasoning and planning task, remains dubious. In this paper, we introduce AlphaLLM for the self-improvements of LLMs, which integrates Monte Carlo Tree Search (MCTS) with LLMs to establish a self-improving loop, thereby enhancing the capabilities of LLMs without additional annotations. Drawing inspiration from the success of AlphaGo, AlphaLLM addresses the unique challenges of combining MCTS with LLM for self-improvement, including data scarcity, the vastness search spaces of language tasks, and the subjective nature of feedback in language tasks. AlphaLLM is comprised of prompt synthesis component, an efficient MCTS approach tailored for language tasks, and a trio of critic models for precise feedback. Our experimental results in mathematical reasoning tasks demonstrate that AlphaLLM significantly enhances the performance of LLMs without additional annotations, showing the potential for self-improvement in LLMs."""
#abstract = r"""We introduce Blink, a new benchmark for multimodal language models (LLMs) that focuses on core visual perception abilities not found in other evaluations. Most of the Blink tasks can be solved by humans "within a blink" (e.g., relative depth estimation, visual correspondence, forensics detection, and multi-view reasoning). However, we find these perception-demanding tasks cast significant challenges for current multimodal LLMs because they resist mediation through natural language. Blink reformats 14 classic computer vision tasks into 3,807 multiple-choice questions, paired with single or multiple images and visual prompting. While humans get 95.70% accuracy on average, Blink is surprisingly challenging for existing multimodal LLMs: even the best-performing GPT-4V and Gemini achieve accuracies of 51.26% and 45.72%, only 13.17% and 7.63% higher than random guessing, indicating that such perception abilities have not "emerged" yet in recent multimodal LLMs. Our analysis also highlights that specialist CV models could solve these problems much better, suggesting potential pathways for future improvements. We believe Blink will stimulate the community to help multimodal LLMs catch up with human-level visual perception."""
#abstract = r"""We propose MeshLRM, a novel LRM-based approach that can reconstruct a high-quality mesh from merely four input images in less than one second. Different from previous large reconstruction models (LRMs) that focus on NeRF-based reconstruction, MeshLRM incorporates differentiable mesh extraction and rendering within the LRM framework. This allows for end-to-end mesh reconstruction by fine-tuning a pre-trained NeRF LRM with mesh rendering. Moreover, we improve the LRM architecture by simplifying several complex designs in previous LRMs. MeshLRM's NeRF initialization is sequentially trained with low- and high-resolution images; this new LRM training strategy enables significantly faster convergence and thereby leads to better quality with less compute. Our approach achieves state-of-the-art mesh reconstruction from sparse-view inputs and also allows for many downstream applications, including text-to-3D and single-image-to-3D generation. Project page: https://sarahweiii.github.io/meshlrm/"""
#abstract = r"""With large language models (LLMs) widely deployed in long content generation recently, there has emerged an increasing demand for efficient long-sequence inference support. However, key-value (KV) cache, which is stored to avoid re-computation, has emerged as a critical bottleneck by growing linearly in size with the sequence length. Due to the auto-regressive nature of LLMs, the entire KV cache will be loaded for every generated token, resulting in low utilization of computational cores and high latency. While various compression methods for KV cache have been proposed to alleviate this issue, they suffer from degradation in generation quality. We introduce TriForce, a hierarchical speculative decoding system that is scalable to long sequence generation. This approach leverages the original model weights and dynamic sparse KV cache via retrieval as a draft model, which serves as an intermediate layer in the hierarchy and is further speculated by a smaller model to reduce its drafting latency. TriForce not only facilitates impressive speedups for Llama2-7B-128K, achieving up to 2.31times on an A100 GPU but also showcases scalability in handling even longer contexts. For the offloading setting on two RTX 4090 GPUs, TriForce achieves 0.108s/tokenx2014only half as slow as the auto-regressive baseline on an A100, which attains 7.78times on our optimized offloading system. Additionally, TriForce performs 4.86times than DeepSpeed-Zero-Inference on a single RTX 4090 GPU. TriForce's robustness is highlighted by its consistently outstanding performance across various temperatures. The code is available at https://github.com/Infini-AI-Lab/TriForce."""
abstract = r"""The intensive computational burden of Stable Diffusion (SD) for text-to-image generation poses a significant hurdle for its practical application. To tackle this challenge, recent research focuses on methods to reduce sampling steps, such as Latent Consistency Model (LCM), and on employing architectural optimizations, including pruning and knowledge distillation. Diverging from existing approaches, we uniquely start with a compact SD variant, BK-SDM. We observe that directly applying LCM to BK-SDM with commonly used crawled datasets yields unsatisfactory results. It leads us to develop two strategies: (1) leveraging high-quality image-text pairs from leading generative models and (2) designing an advanced distillation process tailored for LCM. Through our thorough exploration of quantization, profiling, and on-device deployment, we achieve rapid generation of photo-realistic, text-aligned images in just two steps, with latency under one second on resource-limited edge devices."""
topic_classification_prompt = f"""
Your task is to take an arXiv abstract and classify it into one of the following categories: LLMs, diffusion, computer vision, multimodal.
If you see LLMs or language models in the abstract, classify as LLMs.  If you see computer vision in the abstract, classify as computer vision.
If you see multimodal in abstract, classify as multimodal.  If you see diffusion in the abstract, classify as diffusion.
Now here is the abstract: {abstract}
Think step-by-step about your answer and check your answer before returning it and provide a brief explanation.

Category:
"""
#topic_classification_prompt = 
ans = call_llm(question=topic_classification_prompt, generator=generator_llm,settings=generator_settings,max_new_tokens=24)[len(topic_classification_prompt):]
#pattern = r'(Category|Answer):\s*(\w+)'
pattern = r'(LLMs|diffusion|computer vision|multimodal)'
match = re.search(pattern, ans,re.IGNORECASE)
response =  match.group(1) if match else 'other'
response

'diffusion'

In [279]:
ans

'Category:\n [/INST] I would classify this abstract as "diffusion". The abstract mentions "Stable Diffusion (SD)" as'

In [252]:
# #abstract = r"""JWST is discovering a large population of z>4 supermassive black holes (SMBHs) that are overmassive with respect to the stellar content of their hosts. A previous study developed a physical model to interpret this overmassive population as the result of quasar feedback acting on a compact host galaxy. In this Note, we apply this model to JADES GN 1146115, a dormant supermassive black hole at z=6.7 whose mass is ∼40% of the host's mass in stars and accreting at ∼2% of the Eddington limit. The host has been forming stars at the low rate of ∼1M⊙yr−1 for the past ∼100 Myr. Our model suggests that this galactic system is on the verge of a resurgence of global star formation activity. This transition comes after a period of domination by the effect of its overmassive black hole, whose duration is comparable to typical quasar lifetimes. """
# abstract = r"""Text animation serves as an expressive medium, transforming static communication into dynamic experiences by infusing words with motion to evoke emotions, emphasize meanings, and construct compelling narratives. Crafting animations that are semantically aware poses significant challenges, demanding expertise in graphic design and animation. We present an automated text animation scheme, termed "Dynamic Typography", which combines two challenging tasks. It deforms letters to convey semantic meaning and infuses them with vibrant movements based on user prompts. Our technique harnesses vector graphics representations and an end-to-end optimization-based framework. This framework employs neural displacement fields to convert letters into base shapes and applies per-frame motion, encouraging coherence with the intended textual concept. Shape preservation techniques and perceptual loss regularization are employed to maintain legibility and structural integrity throughout the animation process. We demonstrate the generalizability of our approach across various text-to-video models and highlight the superiority of our end-to-end methodology over baseline methods, which might comprise separate tasks. Through quantitative and qualitative evaluations, we demonstrate the effectiveness of our framework in generating coherent text animations that faithfully interpret user prompts while maintaining readability. Our code is available at: https://animate-your-word.github.io/demo/."""
# #abstract = r"""We introduce Reka Core, Flash, and Edge, a series of powerful multimodal language models trained from scratch by Reka. Reka models are able to process and reason with text, images, video, and audio inputs. This technical report discusses details of training some of these models and provides comprehensive evaluation results. We show that Reka Edge and Reka Flash are not only state-of-the-art but also outperform many much larger models, delivering outsized values for their respective compute class. Meanwhile, our most capable and largest model, Reka Core, approaches the best frontier models on both automatic evaluations and blind human evaluations. On image question answering benchmarks (e.g. MMMU, VQAv2), Core performs competitively to GPT4-V. Meanwhile, on multimodal chat, Core ranks as the second most preferred model under a blind third-party human evaluation setup, outperforming other models such as Claude 3 Opus. On text benchmarks, Core not only performs competitively to other frontier models on a set of well-established benchmarks (e.g. MMLU, GSM8K) but also outperforms GPT4-0613 on human evaluation. On video question answering (Perception-Test), Core outperforms Gemini Ultra. Models are shipped in production at http://chat.reka.ai . A showcase of non cherry picked qualitative examples can also be found at http://showcase.reka.ai ."""
# #abstract = r"""Despite the impressive capabilities of Large Language Models (LLMs) on various tasks, they still struggle with scenarios that involves complex reasoning and planning. Recent work proposed advanced prompting techniques and the necessity of fine-tuning with high-quality data to augment LLMs' reasoning abilities. However, these approaches are inherently constrained by data availability and quality. In light of this, self-correction and self-learning emerge as viable solutions, employing strategies that allow LLMs to refine their outputs and learn from self-assessed rewards. Yet, the efficacy of LLMs in self-refining its response, particularly in complex reasoning and planning task, remains dubious. In this paper, we introduce AlphaLLM for the self-improvements of LLMs, which integrates Monte Carlo Tree Search (MCTS) with LLMs to establish a self-improving loop, thereby enhancing the capabilities of LLMs without additional annotations. Drawing inspiration from the success of AlphaGo, AlphaLLM addresses the unique challenges of combining MCTS with LLM for self-improvement, including data scarcity, the vastness search spaces of language tasks, and the subjective nature of feedback in language tasks. AlphaLLM is comprised of prompt synthesis component, an efficient MCTS approach tailored for language tasks, and a trio of critic models for precise feedback. Our experimental results in mathematical reasoning tasks demonstrate that AlphaLLM significantly enhances the performance of LLMs without additional annotations, showing the potential for self-improvement in LLMs."""

# topic_classification_prompt = f"""
# Your task is to take an arXiv abstract and classify the paper as LLMs (large langugage models) or other
# If you see 'LLMs' or 'language models' in the abstract, classify as 'LLMs'; else classify as 'other'
# Now here is the abstract: {abstract}
# Think step-by-step about your answer and check your answer before returning it and provide a brief explanation.


# Category:
# """
# #topic_classification_prompt = 
# ans = call_llm(question=topic_classification_prompt, generator=generator_llm,settings=generator_settings,max_new_tokens=24)[len(topic_classification_prompt):]
# #pattern = r'(Category|Answer):\s*(\w+)'
# pattern = r'(LLMs|diffusion|computer vision|multimodal)'
# match = re.search(pattern, ans,re.IGNORECASE)
# response =  match.group(1) if match else 'other'
# response

'LLMs'

In [248]:
topic_classification_prompt

'\nYour task is to take an arXiv abstract and classify the paper as LLMs (large langugage models) or other\nIf you see \'LLMs\' or \'language models\' in the abstract, classify as \'LLMs\'; else classify as \'other\'\nNow here is the abstract: Text animation serves as an expressive medium, transforming static communication into dynamic experiences by infusing words with motion to evoke emotions, emphasize meanings, and construct compelling narratives. Crafting animations that are semantically aware poses significant challenges, demanding expertise in graphic design and animation. We present an automated text animation scheme, termed "Dynamic Typography", which combines two challenging tasks. It deforms letters to convey semantic meaning and infuses them with vibrant movements based on user prompts. Our technique harnesses vector graphics representations and an end-to-end optimization-based framework. This framework employs neural displacement fields to convert letters into base shapes 

In [249]:
ans

'Category:\n [/INST] Step 1: Read the abstract carefully\n\nStep 2: Look for any mention of "Large Language Models'

In [241]:
abstract = r"""We introduce Reka Core, Flash, and Edge, a series of powerful multimodal language models trained from scratch by Reka. Reka models are able to process and reason with text, images, video, and audio inputs. This technical report discusses details of training some of these models and provides comprehensive evaluation results. We show that Reka Edge and Reka Flash are not only state-of-the-art but also outperform many much larger models, delivering outsized values for their respective compute class. Meanwhile, our most capable and largest model, Reka Core, approaches the best frontier models on both automatic evaluations and blind human evaluations. On image question answering benchmarks (e.g. MMMU, VQAv2), Core performs competitively to GPT4-V. Meanwhile, on multimodal chat, Core ranks as the second most preferred model under a blind third-party human evaluation setup, outperforming other models such as Claude 3 Opus. On text benchmarks, Core not only performs competitively to other frontier models on a set of well-established benchmarks (e.g. MMLU, GSM8K) but also outperforms GPT4-0613 on human evaluation. On video question answering (Perception-Test), Core outperforms Gemini Ultra. Models are shipped in production at http://chat.reka.ai . A showcase of non cherry picked qualitative examples can also be found at http://showcase.reka.ai ."""
#pattern = r'(LLMs|diffusion|computer vision|multimodal)'
pattern = r'(LLMs)'
match = re.search(pattern, ans,re.IGNORECASE)
response =  match.group(1) if match else 'other'
response

'LLMs'

- Looks like can get Mistral exl2 to classify in .1-.2 seconds

In [161]:
match

<re.Match object; span=(19, 24), match='Other'>

- OK, trying old school route, not useful

In [168]:
# import gensim
# from gensim import corpora
# from gensim.models.ldamodel import LdaModel
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import nltk

# # Download NLTK stopwords
# nltk.download('stopwords')
# nltk.download('punkt')

# # Sample abstract
# #abstract = "Your arXiv abstract text goes here."
# abstract = r"""We introduce Reka Core, Flash, and Edge, a series of powerful multimodal language models trained from scratch by Reka. Reka models are able to process and reason with text, images, video, and audio inputs. This technical report discusses details of training some of these models and provides comprehensive evaluation results. We show that Reka Edge and Reka Flash are not only state-of-the-art but also outperform many much larger models, delivering outsized values for their respective compute class. Meanwhile, our most capable and largest model, Reka Core, approaches the best frontier models on both automatic evaluations and blind human evaluations. On image question answering benchmarks (e.g. MMMU, VQAv2), Core performs competitively to GPT4-V. Meanwhile, on multimodal chat, Core ranks as the second most preferred model under a blind third-party human evaluation setup, outperforming other models such as Claude 3 Opus. On text benchmarks, Core not only performs competitively to other frontier models on a set of well-established benchmarks (e.g. MMLU, GSM8K) but also outperforms GPT4-0613 on human evaluation. On video question answering (Perception-Test), Core outperforms Gemini Ultra. Models are shipped in production at http://chat.reka.ai . A showcase of non cherry picked qualitative examples can also be found at http://showcase.reka.ai ."""

# # Preprocess the text
# def preprocess(text):
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(text.lower())
#     return [token for token in tokens if token not in stop_words and token.isalpha()]

# # Tokenize the abstract
# tokenized_abstract = preprocess(abstract)

# # Create a dictionary representation of the documents
# dictionary = corpora.Dictionary([tokenized_abstract])

# # Create a bag-of-words representation of the documents
# corpus = [dictionary.doc2bow(tokenized_abstract)]

# # Train the LDA model
# lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=15)

# # Get the topic distribution for the abstract
# abstract_topics = lda_model.get_document_topics(corpus[0])

# # Print the topics with their probabilities
# for topic_id, prob in abstract_topics:
#     print(f"Topic ID: {topic_id} - Probability: {prob}")
#     topic = lda_model.print_topic(topic_id)
#     print(topic)

Topic ID: 9 - Probability: 0.9927418231964111
0.062*"models" + 0.046*"core" + 0.046*"reka" + 0.024*"also" + 0.024*"evaluation" + 0.024*"human" + 0.024*"benchmarks" + 0.016*"text" + 0.016*"model" + 0.016*"flash"


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mainuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mainuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# for i, doc in enumerate(docs):
#    #content = docs[index]
#    if i == 0:
#        vector_store = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#    else:
#       vector_store_i = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store

In [36]:
# for index, doc in enumerate(docs):
#    #content = docs[index]
#    if index == 0:
#        vector_store = FAISS.from_texts(doc.page_content, embedder)
#    else:
#       vector_store_i = FAISS.from_texts(doc.page_content, embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store